<a href="https://colab.research.google.com/github/supertime1/Afib_PPG/blob/master/Afib_ECG_125Hz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Introduction

This notebook trains an ECG DNN by using labeled ECG data from "The PhysioNet Computing in Cardiology Challenge 2017" (https://physionet.org/content/challenge-2017/1.0.0/). The ECG DNN model will be used to label the ECG data from MIMIC-III waveform dataset, so as the concurrent PPG data.

The ECG data used in training and validation has the following important attributes:
*   sampling frequency: 300Hz
*   4 lables: Normal (N), AF (A), Other rhythm (O), Noisy (~)
*   length: 9 - 60s with 30s mean.
*   preprosessed: data has been band pass filtered by AliveCor device


Only time length >30s is used in training, since PPG data usually requires 30s for Afib detection.

#2.Setup Environment

In [ ]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import wfdb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model 
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import tensorflow_datasets as tfds
import multiprocessing
from datetime import datetime
import sklearn.metrics
import sklearn
import itertools
import io
print(tf.__version__)

In [ ]:
#run this cell if multiple GPUs are used
tf.debugging.set_log_device_placement(True)

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
tf.test.is_built_with_cuda()

#3.Data Pipeline

##3.1 Load data

###3.1.1 Load training data

In [ ]:
hd_names = []
for name in glob.glob("C:/Users/57lzhang.US04WW4008/Desktop/Afib/Afib data/Afib ECG data/training2017/*.hea"):
  position = name.index('.hea')
  name = name[0:position] #remove the .hea part to comply with the wfdb.rdrecord format
  hd_names.append(name)
print('There are total', len(hd_names), 'records')

In [ ]:
qualified_names = [] #a list of file names that contain ECG Lead I data
for name in hd_names:
  record = wfdb.rdheader(name)
  if record.sig_len >= 9000: #extact only records contrains ECG lead I >30s
    qualified_names.append(name)
print('There are total', len(qualified_names), 'qualified (>30s) records')

In [ ]:
#load label numpy file
df = pd.read_csv(r'C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib data\Afib ECG data\training2017\REFERENCE.csv', sep=',', header=None) 
#create a new name list that only stores the file name
init_labels =[]
new_names = []
for name in qualified_names:
  temp_name = name[-6:] #remove the dir and keep only the file name
  temp_label = df[df[0] == temp_name][1].to_numpy()
  if temp_label != '~':
    init_labels.append(temp_label)
    new_names.append(name)
init_labels = np.array(init_labels)

init_labels[init_labels =='N'] = '0'
init_labels[init_labels =='A'] = '1'
init_labels[init_labels =='O'] = '0'

print('There are total', len(init_labels),'none-nosiy labels')
print('There are total', len(new_names), 'none-nosiy records')

In [ ]:
##read signals
ECG_signals = [] #create a  list to store all  ECG signals
for name in new_names:
  record = wfdb.rdrecord(name)
  ECG_signals.append(record.p_signal)

print('ECG signals len:', len(ECG_signals))

In [ ]:
#A function to split the raw signal data with 30s per segment and keep the label
##source is the raw signal (e.g. ECG_signals) and seg_len = 30s * 300Hz = 9000
def generate_segment_data(source,init_labels,seg_len):
  n=0
  signals =[]
  labels = []
  for signal in source:
    for i in range(int(len(signal)/seg_len)):
      seg = signal[seg_len*i:seg_len*(i+1)]
      label = init_labels[n]
      signals.append(seg)
      labels.append(label)
    n+=1
#convert list into a numpy array and change its dim from (num of records, seg_len, 1) to (num of records, seg_len)
  signals = np.asarray(list(map(lambda x: np.reshape(x,9000),signals)))
  labels = np.asarray(list(map(lambda x: np.reshape(x,1),labels)))

  return signals,labels

In [ ]:
#use generate_segment_data() to generate segments with labels
#After segmentation, more data than previous is generated, because some source data are 60s 
signals, labels = generate_segment_data(ECG_signals,init_labels,9000)
print('signals dim:',signals.shape)
print('labels dim:',labels.shape)

In [ ]:
#add normalization to signals
signals = sklearn.preprocessing.scale(signals)
print('signals dim:',signals.shape)
print('labels dim:',labels.shape)

In [ ]:
#resample the ECG signal
from wfdb import processing
resamp_ECG_signals = []
for i in range(len(signals)):
  resamp_ECG_signal, t = wfdb.processing.resample_sig(signals[i],300,125)
  resamp_ECG_signals.append(resamp_ECG_signal)

In [ ]:
print('Resampled ECG signal dim:', len(resamp_ECG_signals[0]))
print('Resampled ECG signals len:', len(resamp_ECG_signals))

In [ ]:
plt.plot(resamp_ECG_signals[1][:180]*-1,label = "125Hz")
plt.figure()
plt.plot(signals[1][:432]*-1, label="300Hz")
plt.legend()

In [ ]:
#resize the dimension for use in CNN
signals = np.expand_dims(resamp_ECG_signals,axis=1)
signals = np.expand_dims(signals,axis=3)
print('signals dim after resize:',signals.shape)
print('labels dim after resize:',labels.shape)

###3.1.2 Create train and validation set

In [ ]:
###ratio value is between 0 and 1
def slice_dataset(dataset,labels,train_ratio,seed = 10):   #make sure seed is set to a same number for repeatable results
  DATASET_SIZE =len(list(dataset)) #only works in eager mode (e.g. TF version >= 2.0.x)
  train_size = int(train_ratio * DATASET_SIZE)
  val_size = DATASET_SIZE - train_size
  
  np.random.seed(seed=seed)
  np.random.shuffle(dataset)
  train_dataset = dataset[:train_size,:,:]
  val_dataset = dataset[-val_size:,:,:]

  np.random.seed(seed=seed)
  np.random.shuffle(labels)
  train_labels = labels[:train_size,:]
  val_labels = labels[-val_size:,:]

  return train_dataset,val_dataset,train_labels, val_labels

In [ ]:
train_signals, val_signals, train_labels, val_labels  = slice_dataset(signals,labels,0.9)
print("train_dataset dim", train_signals.shape)
print("train_labels dim", train_labels.shape)
print("test_dataset dim", val_signals.shape)
print("test_labels dim", val_labels.shape)

In [ ]:
#check unique labels in train dataset
unique, count = np.unique(train_labels,return_counts=True)
print('There are', count[0], 'No Afib records in training dataset')
print('There are', count[1], 'Afib records in training dataset')

In [ ]:
#check unique labels in train dataset
unique, count = np.unique(val_labels,return_counts=True)
print('There are', count[0], 'No Afib records in validation dataset')
print('There are', count[1], 'Afib records in validation dataset')

###3.1.3 Load test data

In [ ]:
#load test data
test_hd_names = []
for name in glob.glob("C:/Users/57lzhang.US04WW4008/Desktop/Afib/Afib data/Afib ECG data/validation/*.hea"): 
  position = name.index('.hea')
  name = name[0:position] #remove the .hea part to comply with the wfdb.rdrecord format
  test_hd_names.append(name)
print('There are total', len(test_hd_names), 'test records')

In [ ]:
test_qualified_names = [] #a list of file names that contain ECG Lead I data
for name in test_hd_names:
  record = wfdb.rdheader(name)
  if record.sig_len >= 9000: #extact only records contrains ECG lead I >30s
    test_qualified_names.append(name)
print('There are total', len(test_qualified_names), 'qualified (>30s) test records')

In [ ]:
#load label numpy file
df = pd.read_csv(r'C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib data\Afib ECG data\validation\REFERENCE.csv', sep=',', header=None) #'/content/drive/My Drive/training2017/REFERENCE.csv',sep=',', header=None)#
#create a new name list that only stores the file name
test_init_labels =[]
test_new_names = []
for name in test_qualified_names:
  temp_name = name[-6:] #remove the dir and keep only the file name
  temp_label = df[df[0] == temp_name][1].to_numpy()
  if temp_label != '~':
    test_init_labels.append(temp_label)
    test_new_names.append(name)
test_init_labels = np.array(test_init_labels)

test_init_labels[test_init_labels =='N'] = '0'
test_init_labels[test_init_labels =='A'] = '1'
test_init_labels[test_init_labels =='O'] = '0'

print('There are total', len(test_init_labels),'none-nosiy test labels')
print('There are total', len(test_new_names), 'none-nosiy test records')

In [ ]:
##read signals
test_ECG_signals = [] #create a  list to store all  ECG signals
for name in test_new_names:
  record = wfdb.rdrecord(name)
  test_ECG_signals.append(record.p_signal)

print('test ECG signals len:', len(test_ECG_signals))

In [ ]:
#use generate_segment_data() to generate segments with labels
#After segmentation, more data than previous is generated, because some source data are 60s 
test_signals, test_labels = generate_segment_data(test_ECG_signals,test_init_labels,9000)
print('test signals dim:',test_signals.shape)
print('test labels dim:',test_labels.shape)

In [ ]:
#add normalization to signals
test_signals = sklearn.preprocessing.scale(test_signals)
print('After normalization, test signals dim:',test_signals.shape)
print('After normalization, test labels dim:',test_labels.shape)

In [ ]:
#resample the test ECG signal
from wfdb import processing
resamp_test_ECG_signals = []
for i in range(len(test_signals)):
  resamp_test_ECG_signal, t = wfdb.processing.resample_sig(test_signals[i],300,125)
  resamp_test_ECG_signals.append(resamp_test_ECG_signal)

In [ ]:
print('Resampled ECG signal dim:', len(resamp_test_ECG_signals[0]))
print('Resampled ECG signals len:', len(resamp_test_ECG_signals))

In [ ]:
#resize the dimension for use in CNN
test_signals = np.expand_dims(resamp_test_ECG_signals,axis=1)
test_signals = np.expand_dims(test_signals,axis=3)
print('test signals dim after resize:',test_signals.shape)
print('test labels dim after resize:',test_labels.shape)

In [ ]:
#check unique labels
unique, count = np.unique(test_labels,return_counts=True)
print('There are', count[0], 'No Afib records in test dataset')
print('There are', count[1], 'Afib records test dataset')

In [ ]:
#convert test labels to floating point type, so that it can be compared with model output
test_labels = test_labels.flatten()
test_labels = test_labels.astype(float)
type(test_labels[0])

##3.2 Extract, Transform and Load data

###3.2.1 Parallelize Extraction

In [ ]:
train_labels = tf.strings.to_number(train_labels)
train_dataset = tf.data.Dataset.from_tensor_slices((train_signals,train_labels))
val_labels = tf.strings.to_number(val_labels)
val_dataset = tf.data.Dataset.from_tensor_slices((val_signals,val_labels))


### 3.2.2 Parallelize Transformation


In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
#dataset = dataset.map(function, num_parallel_calls = cores)

### 3.2.3 Parallelize Loading

In [ ]:
batch_size = 32
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(2048).repeat().batch(batch_size,drop_remainder=True)
train_dataset = train_dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.repeat().batch(batch_size, drop_remainder=True)

#4.Train Model

##4.1 Build the model



In [ ]:
model = tf.keras.Sequential([
    #1st Conv2D
    tf.keras.layers.Conv2D(8, (1, 1), strides=(1, 1), 
                          activation='relu', input_shape=(1,3750,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #2nd Conv2D
    tf.keras.layers.Conv2D(16, (1, 3), strides=(1, 1),
                          activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #3rd Conv2D
    tf.keras.layers.Conv2D(32, (1, 3), strides=(1, 1),
                          activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #4th Conv2D
    tf.keras.layers.Conv2D(64, (1, 3), strides=(1, 1),
                          activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #5th Conv2D
    tf.keras.layers.Conv2D(16, (1, 1), strides=(1, 1),
                          activation='relu'),
    tf.keras.layers.BatchNormalization(),
    #Full connection layer
    tf.keras.layers.Flatten(),
    #tf.keras.layers.LSTM(50, stateful=True, return_sequences=True),
    #tf.keras.layers.LSTM(10, stateful=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.summary()

##4.2 Define callbacks

###4.2.1 Learning rate scheduler callback

In [ ]:
def decay(epoch):
  if epoch < 30:
    return 1e-3
  elif epoch >= 30 and epoch < 70:
    return 1e-4
  else:
    return 1e-5

In [ ]:
#callback: schedule a learning rate incline iteration
lr_schedule = tf.keras.callbacks.LearningRateScheduler(decay)

###4.2.2 Tensorboard callback

In [ ]:
#callback: tensorboard
log_dir=r"C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib data\logs\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S") +"resnet_045ECG_025PPG"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

###4.2.3 Create a Confusion Matrix callback

In [ ]:
def plot_to_image(figure):
    """
    Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call.
    """
    
    buf = io.BytesIO()
    
    # Use plt.savefig to save the plot to a PNG in memory.
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    
    # Use tf.image.decode_png to convert the PNG buffer
    # to a TF image. Make sure you use 4 channels.
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Use tf.expand_dims to add the batch dimension
    image = tf.expand_dims(image,0)
    
    return image

In [ ]:
class_names = ['NO Afib','Afib']

def plot_confusion_matrix(cm, class_names, normalize=False):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)
    plt.ylim(bottom=1.5,top = -0.5)
    
    if normalize:
      cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 1.5
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j], 
               horizontalalignment="center", 
               verticalalignment='center', 
               color="white" if cm[i, j] > threshold else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [ ]:
def log_confusion_matrix(epoch, logs):
    
    # Use the model to predict the values from the test_images.
    test_pred_raw = model.predict(test_signals)
    
    test_pred = np.where(test_pred_raw > 0.5, 1, 0)
    
    # Calculate the confusion matrix using sklearn.metrics
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    
    figure = plot_confusion_matrix(cm, class_names=class_names, normalize = True)
    cm_image = plot_to_image(figure)
    
    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
#callback: confusion matrix
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

###4.2.4 Checkpoint

In [ ]:
#callback: checkpoint
filepath = r"C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib data\models\resnet-045ECG-p01-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')

## 4.3 Start Training

In [ ]:
#clear history if necessary
tf.keras.backend.clear_session()

#train model
#strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()) ##to overwrite NCCL cross device communication as this is running in Windows
#with strategy.scope():

model = model
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.binary_crossentropy, 
              metrics=['accuracy'])


callbacks_list = [tensorboard_callback, cm_callback, checkpoint, lr_schedule]

#start training
model.fit(train_dataset,
          epochs=200,
          steps_per_epoch = int(len(list(train_signals))/batch_size),
          verbose=1,
          validation_data=val_dataset,
          validation_steps = int(len(list(val_signals))/batch_size),
          callbacks=callbacks_list
          );

## 4.4 Save Model for future evaluation

In [ ]:
os.chdir(r"C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib_ECG data")
model.save('Deep_ECG_125Hz_Normal_Weight.h5')
print("Save model to disk")

#5.Model Evaluation

## 5.1 Load saved model

In [ ]:
os.chdir(r"C:\Users\57lzhang.US04WW4008\Desktop\Afib\Afib_ECG data")
model = tf.keras.models.load_model('Deep_ECG_125Hz_Normal_Weight.h5')

## 5.2 Confusion Matrix

In [ ]:
threshold = 0.006
test_pred_raw = model.predict(test_signals)
test_pred = np.where(test_pred_raw > threshold, 1, 0)
# Calculate the confusion matrix using sklearn.metrics
cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
figure_norm = plot_confusion_matrix(cm, class_names=class_names, normalize=True)
figure_norm.show()
figure = plot_confusion_matrix(cm, class_names=class_names, normalize=False)
figure.show()

## 5.3 F-1 Score

In [ ]:
report = sklearn.metrics.classification_report(test_labels, test_pred)

In [ ]:
print(report)

## 5.4 AUC

In [ ]:
score = sklearn.metrics.roc_auc_score(test_labels, test_pred)

In [ ]:
print(score)

In [ ]:
import sklearn.metrics as metrics
probs = model.predict_proba(test_signals)
preds = probs[:,]
fpr, tpr, threshold = metrics.roc_curve(test_labels, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()